In [1]:
import numpy as np
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [2]:
#mount googledrive
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
#dezipping
!unzip /content/gdrive/MyDrive/UofT/Project/SecondData/Augdata_split.zip -d my_data

Archive:  /content/gdrive/MyDrive/UofT/Project/SecondData/Augdata_split.zip
replace my_data/Augdata_split/test/30kmh/Classimg1036.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [4]:
# location on Google Drive
master_path = '/content/my_data/Augdata_split/'

# Transform Settings - Do not use RandomResizedCrop
transform = transforms.Compose([transforms.Resize((224,224)), 
                                transforms.ToTensor()])

# assumes three folders with 60% training, 20% validation and 20% testing samples
train_dataset = torchvision.datasets.ImageFolder(master_path + 'train', transform=transform)
val_dataset = torchvision.datasets.ImageFolder(master_path + 'val', transform=transform)
test_dataset = torchvision.datasets.ImageFolder(master_path + 'test', transform=transform)

In [5]:
import torch
from torchvision import datasets

def get_images_and_labels(dataset):
    """
    Returns a list of all the images in a torchvision.datasets object
    and a list of all the corresponding labels.

    Args:
        dataset: A torchvision.datasets object.

    Returns:
        A tuple containing two lists: the first list is the list of all the images in the dataset,
        and the second list is the list of all the corresponding labels.
    """
    # Initialize empty lists to store images and labels
    images = []
    labels = []

    # Loop over all data points in the dataset
    for i in range(len(dataset)):
        # Get the image and label for the current data point
        image, label = dataset[i]
        image = image.permute(1, 2, 0)
        img = image.numpy() 
        # Append the image and label to the corresponding lists
        images.append(img
                      )
        labels.append(label)

    return images, labels


In [6]:
train_images, train_labels = get_images_and_labels(train_dataset)
val_images, val_labels = get_images_and_labels(val_dataset)
test_images, test_labels = get_images_and_labels(test_dataset)

In [15]:
type(val_images[0])

numpy.ndarray

In [7]:
import numpy

In [8]:
import cv2
import numpy as np
from sklearn import svm
from sklearn.model_selection import train_test_split
from skimage.feature import hog

# Load the dataset of images and labels
# Here, we assume that the images are stored in a list called "images" and the corresponding labels are stored in a list called "labels"
# Each image is a numpy array of shape (height, width, channels)


# Extract HOG features from each image
hog_features_train = []
for image in train_images:
    # Convert the image to grayscale
    gray_image = cv2.cvtColor(numpy.asarray(image), cv2.COLOR_BGR2GRAY)
    # Compute the HOG features
    features = hog(gray_image, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), block_norm='L2-Hys', transform_sqrt=True)
    hog_features_train.append(features)

hog_features_val = []
for image in val_images:
    # Convert the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Compute the HOG features
    features = hog(gray_image, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), block_norm='L2-Hys', transform_sqrt=True)
    hog_features_val.append(features)


# Create an SVM classifier with a linear kernel
clf = svm.SVC(kernel='linear', C=1)




In [9]:
# Train the classifier on the training set
clf.fit(hog_features_train, train_labels)

SVC(C=1, kernel='linear')

In [ ]:
print("Accuracy:", clf.score(hog_features_train, train_labels))

In [ ]:

# Predict the labels of the test set
y_pred = clf.predict(X_test)

# Print the accuracy score of the classifier
print("Accuracy:", clf.score(hog_features_train, train_labels)